In [ ]:
!pip install docling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.1/145.1 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.8/277.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from docling.document_converter import DocumentConverter

source = "/content/Bangladesh-National-Building-Code-chap1.pdf"
converter = DocumentConverter()
result = converter.convert(source)

# This gives you the full hierarchical JSON
structured_json = result.document.export_to_dict()

# Now you can programmatically find a specific section
# (Example: Accessing the title of the first identified section)
print(structured_json["texts"][0]["text"])

[INFO] 2026-01-12 12:35:04,589 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-12 12:35:04,595 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-12 12:35:04,601 [RapidOCR] download_file.py:68: Initiating download: https://www.modelscope.cn/models/RapidAI/RapidOCR/resolve/v3.5.0/torch/PP-OCRv4/det/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-12 12:35:07,267 [RapidOCR] download_file.py:82: Download size: 13.83MB
[INFO] 2026-01-12 12:35:08,093 [RapidOCR] download_file.py:95: Successfully saved to: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-12 12:35:08,095 [RapidOCR] main.py:50: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2026-01-12 12:35:08,465 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2026-01-12 12:35:08,467 [RapidOCR] device_config.py:50: Using CPU device
[INFO] 2026-01-12 12:35:08,468 [RapidOCR] download_file.py:68: Initiating download: https://

2622


In [ ]:
import json
output_filename = "bnbc-chap1.json"

with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(structured_json, f, ensure_ascii=False, indent=4)

print(f"Success! Your structured data is saved in {output_filename}")

Success! Your structured data is saved in bnbc-chap1.json


In [ ]:
!pip install PyMuPDF


In [ ]:
import json
import re
from pathlib import Path
import fitz  # PyMuPDF


# ----------------------------
# Config (edit these paths)
# ----------------------------
INPUT_DOCLING_JSON = "/content/bnbc-chap1.json"
INPUT_PDF = "/content/Bangladesh-National-Building-Code-chap1.pdf"

OUTPUT_DIR = "/content/structured_out"
IMAGES_DIR_NAME = "images"
DPI = 200
IMAGE_FORMAT = "jpg"


# ----------------------------
# Helpers: Docling JSON access
# ----------------------------
CLAUSE_RE = re.compile(r"^\s*(\d+(?:\.\d+)*)\s+(.*\S)\s*$")


def normalize_ws(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "")).strip()


def load_docling_json(path: str) -> dict:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def resolve_ref(doc: dict, ref: str):
    """
    ref examples:
      '#/texts/93'
      '#/tables/0'
      '#/pictures/2'
      '#/groups/5'
    """
    m = re.match(r"^#/(texts|tables|pictures|groups)/(\d+)$", ref)
    if not m:
        return None, None, None
    kind, idx = m.group(1), int(m.group(2))
    return kind, idx, doc[kind][idx]


def ref_to_index(ref: str) -> int:
    # '#/texts/31' -> 31
    return int(ref.split("/")[-1])


def extract_caption(doc: dict, captions) -> str:
    """
    captions is typically a list like [{'$ref': '#/texts/93'}]
    """
    if not captions:
        return ""
    parts = []
    for c in captions:
        r = c.get("$ref")
        if not r:
            continue
        kind, _, obj = resolve_ref(doc, r)
        if kind == "texts":
            t = normalize_ws(obj.get("text") or "")
            if t:
                parts.append(t)
    return " ".join(parts).strip()


def walk_body_in_reading_order(doc: dict):
    """
    Body children contains a mixture of texts/tables/pictures and also '#/groups/*'.
    We DFS into groups so we don't miss content.
    """
    body = doc.get("body") or {}
    stack = []

    def push_children(children):
        for child in reversed(children or []):
            stack.append(child)

    push_children(body.get("children"))

    while stack:
        item = stack.pop()
        if not isinstance(item, dict) or "$ref" not in item:
            continue

        ref = item["$ref"]
        kind, _, obj = resolve_ref(doc, ref)

        if kind == "groups":
            push_children(obj.get("children"))
            continue

        yield ref


# ----------------------------
# Tables (FIXED)
# ----------------------------
def table_rows_robust(table_obj: dict):
    """
    Robustly converts Docling tables into rows[][].

    Handles:
      A) data["grid"][r][c] is a dict with "text"
      B) data["grid"][r][c] is an int index into data["table_cells"]
      C) missing/empty grid -> reconstruct from table_cells span metadata
    """
    data = table_obj.get("data") or {}
    grid = data.get("grid") or []
    table_cells = data.get("table_cells") or []

    num_rows = int(data.get("num_rows") or (len(grid) if grid else 0))
    num_cols = int(data.get("num_cols") or (len(grid[0]) if grid and grid[0] else 0))

    # Case A: grid holds dict cells directly
    if grid and grid[0] and isinstance(grid[0][0], dict):
        rows = []
        for r in grid:
            row = []
            for cell in r:
                if isinstance(cell, dict):
                    row.append(normalize_ws(cell.get("text", "")))
                else:
                    row.append("")
            rows.append(row)
        return {"num_rows": num_rows, "num_cols": num_cols, "rows": rows}

    # Case B: grid holds ints referencing table_cells
    if grid and grid[0] and isinstance(grid[0][0], int):
        rows = []
        for r in grid:
            row = []
            for idx in r:
                if isinstance(idx, int) and 0 <= idx < len(table_cells):
                    row.append(normalize_ws(table_cells[idx].get("text", "")))
                else:
                    row.append("")
            rows.append(row)
        return {"num_rows": num_rows, "num_cols": num_cols, "rows": rows}

    # Case C: reconstruct from table_cells span metadata
    mat = [["" for _ in range(num_cols)] for _ in range(num_rows)]
    for cell in table_cells:
        txt = normalize_ws(cell.get("text", ""))
        r0 = cell.get("start_row_offset_idx", 0)
        r1 = cell.get("end_row_offset_idx", r0 + 1)
        c0 = cell.get("start_col_offset_idx", 0)
        c1 = cell.get("end_col_offset_idx", c0 + 1)
        for rr in range(r0, r1):
            for cc in range(c0, c1):
                if 0 <= rr < num_rows and 0 <= cc < num_cols:
                    mat[rr][cc] = txt

    return {"num_rows": num_rows, "num_cols": num_cols, "rows": mat}


def build_tables_json(doc: dict) -> list:
    out = []
    for i, tbl in enumerate(doc.get("tables", [])):
        provs = tbl.get("prov") or []
        page_no = provs[0].get("page_no") if provs else None
        bbox = provs[0].get("bbox") if provs else None
        caption = extract_caption(doc, tbl.get("captions"))

        t = table_rows_robust(tbl)  # <-- fixed

        out.append(
            {
                "table_id": f"table_{i:04d}",
                "page_no": page_no,
                "bbox": bbox,
                "caption": caption,
                "num_rows": t["num_rows"],
                "num_cols": t["num_cols"],
                "rows": t["rows"],
            }
        )
    return out


# ----------------------------
# Images (figures) via PDF crop
# ----------------------------
def bbox_to_fitz_rect(bbox: dict, page_height: float) -> fitz.Rect | None:
    """
    Docling bbox has coord_origin, often 'BOTTOMLEFT'.
    PyMuPDF uses TOPLEFT origin.
    """
    l = bbox.get("l")
    t = bbox.get("t")
    r = bbox.get("r")
    b = bbox.get("b")
    if None in (l, t, r, b):
        return None

    origin = (bbox.get("coord_origin") or "TOPLEFT").upper()

    if origin == "BOTTOMLEFT":
        # Convert y coords from bottom-origin to top-origin
        y0 = page_height - t
        y1 = page_height - b
    else:
        y0 = t
        y1 = b

    x0, x1 = sorted([l, r])
    y0, y1 = sorted([y0, y1])
    return fitz.Rect(x0, y0, x1, y1)


def extract_figures_from_pdf(doc: dict, pdf_path: str, out_dir: str, dpi: int, image_format: str):
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    pdf = fitz.open(pdf_path)
    images_meta = []

    for i, pic in enumerate(doc.get("pictures", [])):
        provs = pic.get("prov") or []
        if not provs:
            continue

        prov = provs[0]  # usually only one
        page_no = prov.get("page_no")
        bbox = prov.get("bbox")
        if not page_no or not bbox:
            continue

        page_index = page_no - 1
        if page_index < 0 or page_index >= pdf.page_count:
            continue

        page = pdf.load_page(page_index)
        page_rect = page.rect

        clip = bbox_to_fitz_rect(bbox, page_height=page_rect.height)
        if clip is None:
            continue

        clip = clip & page_rect  # clamp to page
        caption = extract_caption(doc, pic.get("captions"))

        zoom = dpi / 72.0
        pix = page.get_pixmap(matrix=fitz.Matrix(zoom, zoom), clip=clip, alpha=False)

        fname = f"figure_{i:04d}_p{page_no}.{image_format}"
        fpath = out_dir / fname
        pix.save(str(fpath))

        images_meta.append(
            {
                "figure_id": f"figure_{i:04d}",
                "page_no": page_no,
                "bbox": bbox,
                "caption": caption,
                "file": str(fpath),
            }
        )

    pdf.close()
    return images_meta


# ----------------------------
# Clause tree
# ----------------------------
def build_clause_tree(doc: dict):
    """
    Builds nodes keyed by clause id (e.g., '5.5', '5.5.1', '5.5.1.1').
    Figures are stored ONLY as captions (plus id).
    Tables are referenced (plus caption) but table contents are in a separate JSON.
    """
    nodes = {}
    root_id = "ROOT"
    nodes[root_id] = {"id": root_id, "title": "", "text": "", "children": [], "tables": [], "figures": []}
    current_id = root_id

    def ensure_node(cid: str):
        if cid not in nodes:
            nodes[cid] = {"id": cid, "title": "", "text": "", "children": [], "tables": [], "figures": []}

    def parent_id(cid: str) -> str:
        parts = cid.split(".")
        return root_id if len(parts) <= 1 else ".".join(parts[:-1])

    def add_child(pid: str, cid: str):
        if cid not in nodes[pid]["children"]:
            nodes[pid]["children"].append(cid)

    for ref in walk_body_in_reading_order(doc):
        kind, idx, obj = resolve_ref(doc, ref)

        if kind == "texts":
            text = (obj.get("text") or "").strip()
            if not text:
                continue

            m = CLAUSE_RE.match(text)
            if m:
                cid = m.group(1)
                rest = m.group(2).strip()

                ensure_node(cid)
                pid = parent_id(cid)
                ensure_node(pid)
                add_child(pid, cid)

                depth = len(cid.split("."))

                # Heuristic:
                # - depth <= 3: likely headings (section/subsection), store rest as title
                # - depth >= 4: likely “numbered requirement sentence”, store rest as body text
                if depth <= 3:
                    if not nodes[cid]["title"]:
                        nodes[cid]["title"] = rest
                else:
                    nodes[cid]["text"] += rest + "\n"

                current_id = cid
            else:
                nodes[current_id]["text"] += text + "\n"

        elif kind == "tables":
            caption = extract_caption(doc, obj.get("captions"))
            nodes[current_id]["tables"].append({"table_id": f"table_{idx:04d}", "caption": caption})

        elif kind == "pictures":
            caption = extract_caption(doc, obj.get("captions"))
            nodes[current_id]["figures"].append({"figure_id": f"figure_{idx:04d}", "caption": caption})

    # cleanup whitespace
    for nid in nodes:
        nodes[nid]["text"] = nodes[nid]["text"].strip()

    return {"root": root_id, "nodes": nodes}


# ----------------------------
# Optional: retrieval helper
# ----------------------------
def collect_text_recursive(structured: dict, clause_id: str) -> str:
    """
    Returns clause text + all descendants' text (useful when user requests a whole section).
    """
    nodes = structured["nodes"]
    if clause_id not in nodes:
        return ""

    n = nodes[clause_id]
    chunks = []

    if clause_id != "ROOT" and n["title"]:
        chunks.append(f"{clause_id} {n['title']}".strip())
    if n["text"]:
        chunks.append(n["text"])

    for fig in n.get("figures", []):
        if fig.get("caption"):
            chunks.append(fig["caption"])
    for tbl in n.get("tables", []):
        if tbl.get("caption"):
            chunks.append(tbl["caption"])

    for child in n.get("children", []):
        child_txt = collect_text_recursive(structured, child)
        if child_txt:
            chunks.append(child_txt)

    return "\n".join(chunks).strip()


# ----------------------------
# Main
# ----------------------------
def main():
    out_base = Path(OUTPUT_DIR)
    out_base.mkdir(parents=True, exist_ok=True)
    images_dir = out_base / IMAGES_DIR_NAME

    doc = load_docling_json(INPUT_DOCLING_JSON)

    # 1) Clause tree (with figure captions only)
    clauses_struct = build_clause_tree(doc)

    # 2) Tables as rows (FIXED)
    tables_struct = build_tables_json(doc)

    # 3) Figures as JPG + metadata
    images_struct = extract_figures_from_pdf(
        doc=doc,
        pdf_path=INPUT_PDF,
        out_dir=str(images_dir),
        dpi=DPI,
        image_format=IMAGE_FORMAT,
    )

    # Save outputs
    clauses_path = out_base / "structured_clauses.json"
    tables_path = out_base / "structured_tables.json"
    images_path = out_base / "structured_images.json"

    clauses_path.write_text(json.dumps(clauses_struct, ensure_ascii=False, indent=2), encoding="utf-8")
    tables_path.write_text(json.dumps(tables_struct, ensure_ascii=False, indent=2), encoding="utf-8")
    images_path.write_text(json.dumps(images_struct, ensure_ascii=False, indent=2), encoding="utf-8")

    # Example usage: get all text for a section including subclauses
    example_section = next((k for k in clauses_struct["nodes"].keys() if k not in ("ROOT",) and k.count(".") == 1), None)
    if example_section:
        print("\nExample section:", example_section)
        print(collect_text_recursive(clauses_struct, example_section)[:800], "...\n")

    print("Saved:")
    print(" -", clauses_path)
    print(" -", tables_path)
    print(" -", images_path)
    print("Images saved under:", images_dir)


if __name__ == "__main__":
    main()



Example section: 1.1
1.1 Scope
This  Part  of  the  Code  puts  forward  classification  of  buildings  based  on  occupancy  or nature  of  use  and  deals  with  the  general  and  specific  requirements  of  each  of  the occupancy  groups.  Fire  resistance  requirements  are  expressed  in  terms  of  type  of construction which shall conform to the specified fire-resistive properties. ...

Saved:
 - /content/structured_out/structured_clauses.json
 - /content/structured_out/structured_tables.json
 - /content/structured_out/structured_images.json
Images saved under: /content/structured_out/images
